In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# python 3 version

##匯入資源
import urllib.request#python3:http://stackoverflow.com/questions/30261860/python-3-attributeerror-module-object-has-no-attribute-path-using-urll
import urllib.parse
import json
import gzip
import datetime #http://stackoverflow.com/questions/6288892/convert-datetime-format
import re
import calendar

print('交易日期')

##事先告知項目
print("”＊“ 代表非必填、“進口牛蒡“會拿掉處理")

##設定

#關於enddate
fmt = '%Y.%m.%d'
d = datetime.datetime.today()#http://stackoverflow.com/questions/4998629/python-split-string-with-multiple-delimiters
year = int(re.split('[.]', d.strftime(fmt))[0]) -1911
month = re.split('[.]', d.strftime(fmt))[1]
day = re.split('[.]', d.strftime(fmt))[2]
enddate = "%d.%s.%s" % (year, month, day)
yearslist = range(85, year+1)
#關於startdate
day = '01'
startdate = "%d.%s.%s" % (year, month, day)#第一次跑的時候，只跑到那個月的第一天，之後就一直跑那個月的組共資料

#關於時間年月日
months = {1:'01', 2:'02', 3:'03', 4:'04', 5:'05', 6:'06', 7:'07', 8:'08', 9:'09', 10:'10', 11:'11', 12:'12'}
monthsdays = {1:'31', 2:'28', 3:'31', 4:'30', 5:'31', 6:'30', 7:'31', 8:'31', 9:'30', 10:'31', 11:'30', 12:'31'}
monthsdays2 = monthsdays.copy()#http://stackoverflow.com/questions/2465921/how-to-copy-a-dictionary-and-only-edit-the-copy
monthsdays2[2] = '29'#monthsdays2 = {1:31, 2:29, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

#用一個function，每跑一次函數時間就會往前移動一個月
#關於startdate要比enddate少一個月，且要計算到上一個月的第一天
#因為會一直往前推一個月，可能會碰到閏年的2月，要小心有29天，另外要擔心如果要跨到上一年時的情況
def runmonthcycle(year, month, day):#https://www.codecademy.com/forum_questions/55cb254ee39efe13fd000087
    #先處理如果是一月的話
    if int(month) == 1:
        year -= 1
        month = months[12]
        day = monthsdays[12]
    #是閏年
    elif calendar.isleap(year+1911) == True:#http://stackoverflow.com/questions/11621740/how-to-determine-whether-a-year-is-a-leap-year-in-python
        day = monthsdays2[int(month)-1]
        month = months[int(month)-1]
    #不是閏年
    else:
        day = monthsdays[int(month)-1]
        month = months[int(month)-1]
    enddate = "%d.%s.%s" % (year, month, day)
    day = '01'
    startdate = "%d.%s.%s" % (year, month, day)
    #重新計算year, month, day
    #關於enddate
    return startdate, enddate, year, month, day

#關於產地
uriba = {1:'台北一', 2:'台北二', 3:'三重市', 4:'台中市', 5:'高雄市', 6:'鳳山市', 7:'桃園縣'}#6不是'鳳山區'
uribalist = range(7)
crop = '牛蒡'
top = '700'#raw_input('請輸入最大資料擷取數量: ')#一天的資料約有7~9站，預設1000筆資料
trandic = {'交易日期':'dat', '作物代號':'crs', '作物名稱':'cro', '市場名稱':'mar', '市場代號':'mas', '上價':'pru', '中價':'pro', '下價':'prd', '平均價':'prm', '交易量':'trv'}
market = ''

#要求輸入資料
def rundatainputcycle():
    marketinput = input('＊請擇一輸入： 1:台北一, 2:台北二, 3:三重市, 4:台中市, 5:高雄市, 6:鳳山區, 7:桃園縣 或是不填寫 > ')
    if len(str(marketinput)) == 0:
        market = ''
    elif len(str(marketinput)) ==1 and 1 <= int(marketinput) <= 7:
        market = uriba[int(uribalist[int(marketinput)-1]+1)]
    else:
        print('請重新輸入：1:台北一, 2:台北二, 3:三重市, 4:台中市, 5:高雄市, 6:鳳山區, 7:桃園縣 或是不填寫 > ')
        rundatainputcycle()
    return market
market = rundatainputcycle()

#匯入資料
def rundatacycle(market, startdate, enddate):
    url = 'http://m.coa.gov.tw/OpenData/FarmTransData.aspx?$top=' + top + '&$skip=0&' + urllib.parse.urlencode({'crop': crop}) + '&StartDate=' + startdate + '&EndDate=' + enddate
    if market != '':
        url += '&' + urllib.parse.urlencode({'Market': market})
    else:
        url = url
    data = urllib.request.urlopen(url).read().decode('utf-8')#http://stackoverflow.com/questions/28906859/module-has-no-attribute-urlencode
    result = json.loads(data)
    #改變資料結構
    for subdata in result:#subdata = {'下價': '32', '作物代號': 'SM1', '交易量': '620', '中價': '32', '市場名稱': '台北二', '平均價': '32', '作物名稱': '牛蒡', '上價': '32', '交易日期': '105.04.06', '市場代號': '104'}
        for ta, tb in trandic.items():#這個就沒有迴圈的效果
            subdata[tb] = subdata.pop(ta)
    return result
gouba_data = rundatacycle(market, startdate, enddate)

#開始按照月份把資料載下來，從今天所屬的這個月，一直自動存到資料的最開頭101.01.01
#先存這個月的資料出去

def savedata(market, startdate, enddate, year, month, day):
    #開始存資料
    global gouba_data
    with open(str(year) + month + '.json', 'w') as fout:#關於global和local變數的問題 http://stackoverflow.com/questions/10851906/python-3-unboundlocalerror-local-variable-referenced-before-assignment
        json.dump(gouba_data, fout)
        print(str(year) + month + 'json_saved')
    startdate, enddate, year, month, day= runmonthcycle(year, month, day)
    gouba_data = rundatacycle(market, startdate, enddate)
#     print(str(year), month, day)#test
    if year >= 101:
        savedata(market, startdate, enddate, year, month, day)
    return gouba_data, market, startdate, enddate, year, month, day
savedata(market, startdate, enddate, year, month, day)
        
        
"""
迴圈偵測，跑過的跳過，偵測檔案如果有存在，就跳過跑下一輪的迴圈
切成好幾個迴圈跑
等待時間，之後繼續跑迴圈

"""

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import csv
import json
import re
import os
import zipfile#http://stackoverflow.com/questions/7806563/how-to-unzip-a-file-with-python-2-4
from itertools import islice

#設定資料
newdata = {}
title = ['crs', 'mas', 'dat', 'pru', 'pro', 'prd', 'prm', 'trv']
market = {'104':'台北二', '109':'台北一', '241':'三重市', '260':'宜蘭市', '420':'豐原鄉', '512':'永靖鄉', '514':'溪湖鎮', '540':'南投市', '648':'西螺鎮', '900':'屏東市', '400':'台中市', '800':'高雄市', '830':'鳳山市', '338':'桃園縣', '930':'台東市', '950':'花蓮市'}
months = {1:'01', 2:'02', 3:'03', 4:'04', 5:'05', 6:'06', 7:'07', 8:'08', 9:'09', 10:'10', 11:'11', 12:'12'}
for i in range(85, 101):
    for m in range(1, 13):
        newdata[str(i)+months[m]] = []

#解壓縮檔案
pre = '101maeni/'
def unzip(zipFilePath, destDir):
    zfile = zipfile.ZipFile(zipFilePath)
    for name in zfile.namelist():
        (dirName, fileName) = os.path.split(name)
        if fileName == '':
            # directory
            newDir = destDir + '/' + dirName
            if not os.path.exists(newDir):
                os.mkdir(newDir)
        else:
            # file
            fd = open(destDir + '/' + name, 'wb')
            fd.write(zfile.read(name))
            fd.close()
    zfile.close()
for zipdata in newdata.keys():
    unzip(pre+zipdata+'.csv',pre)

#每次載入一個新資料，只想跑一次那筆資料就把所有月份的資料都存下來

#轉存csv為json，從中擷取出資料
for y in range(85, 101):
    year = int(y)
    #判定資料的年份
    if int(y) < 100:
        name = '0' + str(y)
    else:
        name = str(y)
    dataurl = '101maeni/' + name + '.csv'
    #打開檔案
    with open(dataurl, 'r') as fin:
        AQdata = fin.read()
    #把資料換行的時候，當作是切成不同筆資料的分界點，成為多行資料形成的list，一行視為一筆資料
    rowslist = AQdata.split('\n')
    #把每個單行資料切成一個cell一個cell，每一行變成一個list，一個cell為一筆資料
    for rows in islice(rowslist, 1, None):#rows = crs,mas,dat,pru,pro,prd,prm,trv 等等，型態為str，而row = [crs,mas,dat,pru,pro,prd,prm,trv]，型態為list
        #http://stackoverflow.com/questions/10079216/skip-first-entry-in-for-loop-in-python
        #len(rowslist) 有五萬多筆
        row = rows.split(',')#要做切開的動作才會變成list
        row[0] = row[0].replace(' ', '')#有空白在當中，會影響辨識，所以要拿掉
        month = re.split('[.]', row[2])[1]
        data = {}
        #開始做資料轉存
        if row[0] == 'SM1' or row[0] == 'SM9':
            for i in range(8):
                data.update({title[i]:str(row[i])})
            if row[0] == "SM1":
                data['cro'] = '牛蒡'
            elif row[0] == "SM9":
                data['cro'] = '牛蒡-進口'
            for i in range(len(list(market))):#判別他屬於哪一個市場
                if str(row[1]) == list(market.keys())[i]:
                    data['mar'] = market[str(row[1])]
            newdata[str(year)+months[int(month)]].append(data)
    #開始存資料，上面的5萬多筆資料跑好後，按照存到哪一筆dictionary就存檔不同檔案
    for i in range(1, 13):#存入12個月份的資料
        with open(str(year) + months[i] + '.json', 'w') as save:
            json.dump(newdata[str(int(year))+months[i]], save)
            print(str(int(year))+months[i]+'json_saved')

# print(newdata)
# print(len(newdata))

